<h1>ASL Detector Training</h1>
<p>Created by Alex Pereira</p>

<h2>Preprocessing</h2>

In [49]:
# Import Libraries
import os
import numpy as np
import tensorflow as tf
from   tensorflow import keras
from   tensorflow.keras.utils  import to_categorical
from   sklearn.model_selection import train_test_split

In [50]:
# Gets variables for data collection
DATA_PATH    = os.path.join("MP_Data")
action_list  = np.load("actions.npy")  # List of gestures
num_videos   = 30  # Number of videos
video_frames = 30  # Frames per video

In [51]:
# Creates a label map
label_map = {label:num for num, label in enumerate(action_list)}

In [52]:
sequences, labels = [], []

# 
for action in action_list:
    # 
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # 
        window = []

        # 
        for num_frame in range(0, video_frames):
            frame = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(num_frame)))
            window.append(frame)

        #
        sequences.append(window)

        # 
        labels.append(label_map[action])

In [53]:
# Creates a numpy array
X = np.array(sequences)

# Stores the labels in an array
y = to_categorical(labels).astype(int)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

<h2>Model Creation</h2>

In [55]:
# Import model building components
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [56]:
# Makes a sequential model
model = Sequential()

In [57]:
# Adds layers to the model
model.add(LSTM(64,  return_sequences = True , activation = "relu", input_shape = (30,1662)))
model.add(LSTM(128, return_sequences = True , activation = "relu"))
model.add(LSTM(64,  return_sequences = False, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(action_list.shape[0], activation = "softmax"))

# Print a model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [58]:
# Defines the optimizer
customOptimizer = keras.optimizers.Adam()

# Defines the loss function
customLoss      = keras.losses.CategoricalCrossentropy()

# Defines adjustments to the learning rate
def scheduler(epoch, lr):
    adj_lr = lr * tf.math.exp(-0.05)
    
    # Afer 10 epochs the learning rate starts changing
    if epoch < 100:
        return lr
    else:
        return adj_lr

# Defines the callbacks to include
my_callbacks = [
    # tf.keras.callbacks.LearningRateScheduler(scheduler),
    # tf.keras.callbacks.ModelCheckpoint(filepath = "asl_accuracy/asl_detection.acc.{epoch:02d}-{cat_accuracy:.2f}.h5", monitor = "val_cat_accuracy", mode = "max", save_best_only = True),
    # tf.keras.callbacks.ModelCheckpoint(filepath = "asl_loss/asl_detection.loss.{epoch:02d}-{val_loss:.2f}.h5", monitor = "loss", mode = "min", save_best_only = True),
    tf.keras.callbacks.EarlyStopping(patience = 1000, monitor = "cat_accuracy"),
    tf.keras.callbacks.TensorBoard(log_dir  = "Logs")
]

# Creates customized metrics
custom_metrics = [
    tf.keras.metrics.CategoricalAccuracy(name = "cat_accuracy", dtype = None)
]

# Compile the model
model.compile(optimizer = customOptimizer, loss = customLoss, metrics = custom_metrics, jit_compile = True)

In [60]:
# Trains the model
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5000, callbacks = my_callbacks, use_multiprocessing = True)

Epoch 1/5000
3/3 [==============================] - 1s 169ms/step - loss: 0.9758 - cat_accuracy: 0.5059 - val_loss: 1.0233 - val_cat_accuracy: 0.4000
Epoch 2/5000
3/3 [==============================] - 1s 185ms/step - loss: 0.9655 - cat_accuracy: 0.5294 - val_loss: 0.8986 - val_cat_accuracy: 0.6000
Epoch 3/5000
3/3 [==============================] - 1s 187ms/step - loss: 0.9071 - cat_accuracy: 0.5294 - val_loss: 0.8589 - val_cat_accuracy: 0.4000
Epoch 4/5000
3/3 [==============================] - 1s 187ms/step - loss: 0.9725 - cat_accuracy: 0.4824 - val_loss: 1.2485 - val_cat_accuracy: 0.4000
Epoch 5/5000
3/3 [==============================] - 1s 183ms/step - loss: 0.9971 - cat_accuracy: 0.4588 - val_loss: 0.8619 - val_cat_accuracy: 0.2000
Epoch 6/5000
3/3 [==============================] - 1s 179ms/step - loss: 0.8647 - cat_accuracy: 0.5647 - val_loss: 0.8779 - val_cat_accuracy: 0.6000
Epoch 7/5000
3/3 [==============================] - 0s 172ms/step - loss: 0.8228 - cat_accuracy: 0.5